In [1]:
import numpy as np 
import pandas as pd 
import cv2
from PIL import Image
import matplotlib.pyplot as plt
import pandas as pd
from skimage.transform import resize
import tensorflow as tf
from keras.callbacks import ModelCheckpoint, EarlyStopping ,ReduceLROnPlateau
from keras.layers.merge import concatenate
from keras import backend as K
from keras.layers.core import Lambda, RepeatVector, Reshape
from keras.models import Model
from keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, concatenate, Conv2DTranspose, BatchNormalization, Dropout 
import os
from keras.applications.vgg16 import preprocess_input

In [2]:
train_images_path = "../input/assi4-dataset/resized_training/resized_training" 
train_mask_path = "../input/assi4-dataset/resized_training_mask/resized_training_mask"
train_metadata_path = "../input/assi4-dataset/ISIC-2017_Training_Data_metadata.csv"

val_images_path = "../input/assi4-dataset/resized_val/resized_val"
val_mask_path = "../input/assi4-dataset/resized_val_mask/resized_val_mask"
val_metadata_path = "../input/assi4-dataset/ISIC-2017_Validation_Data_metadata.csv"

test_images_path = "../input/assi4-dataset/resized_test"
test_mask_path = "../input/assi4-dataset/resized_test_mask/resized_test_mask"
test_metadata_path = "../input/assi4-dataset/ISIC-2017_Test_v2_Data_metadata.csv"


In [3]:
def read_metadata(path):
    metadata_Dataframe = pd.read_csv(path)
    metadata_Dataframe.head()
    return metadata_Dataframe

In [4]:
def load_images_from_folder(path,ids):
    images = []
    for imageId in ids:
        images.append( cv2.resize(cv2.imread(os.path.join(path,imageId+'.jpg')), (256,256), interpolation = cv2.INTER_AREA) )
    print('images load done')
    return np.array(images)

In [5]:
def load_masks_from_folder(path,ids):
    images = []
    for imageId in ids:
        img = cv2.imread(os.path.join(path,imageId+'.png'))
        img = tf.image.resize(img, (256,256), method='nearest')
        img = tf.image.rgb_to_grayscale(img)/255
        images.append(img)
    print('images load done')
    return np.array(images)

In [6]:
train_metadata = read_metadata(train_metadata_path)
val_metadata = read_metadata(val_metadata_path)
#test_metadata = read_metadata(test_metadata_path)
train_images_ids = train_metadata['image_id']
val_images_ids = val_metadata['image_id']
#test_images_ids = test_metadata['image_id']


train_images = load_images_from_folder(train_images_path,train_images_ids)
val_images = load_images_from_folder(val_images_path,val_images_ids)

In [7]:
train_masks = load_masks_from_folder(train_mask_path,train_images_ids)
val_masks = load_masks_from_folder(val_mask_path,val_images_ids)

In [8]:
plt.figure()
plt.imshow(train_images[1])
plt.figure()
plt.imshow(train_masks[1])
plt.figure()
plt.imshow(val_images[1])
plt.figure()
plt.imshow(val_masks[1])
train_images.shape
val_masks.shape

In [9]:
callbacks = [
    EarlyStopping(patience=5, verbose=1),
    #ReduceLROnPlateau(factor=0.8, patience=2, min_lr=0.00001, verbose=1),
    ModelCheckpoint('tgs-salt-model-v1.h5', verbose=1, save_best_only=True)
]

In [10]:
from keras import backend as K

def jacard_coef(y_true, y_pred):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (intersection + 1.0) / (K.sum(y_true_f) + K.sum(y_pred_f) - intersection + 1.0)


def iou_coef(y_true, y_pred, smooth=10):
    print(y_true)
    print(y_pred)
    intersection = K.sum(K.abs(y_true * y_pred), axis=[1,2,3])
    union = K.sum(y_true,[1,2,3])+K.sum(y_pred,[1,2,3])-intersection
    iou = K.mean((intersection + smooth) / (union + smooth), axis=0)
    return iou

def jaccard_distance_loss(y_true, y_pred, smooth=1):
    intersection = K.sum(K.abs(y_true * y_pred), axis=-1)
    sum_ = K.sum(K.abs(y_true) + K.abs(y_pred), axis=-1)
    jac = (intersection + smooth) / (sum_ - intersection + smooth)
    return (1 - jac) * smooth

def jacard_coef_loss(y_true, y_pred):
    return -iou_coef(y_true, y_pred)  # -1 ultiplied as we want to minimize this value as loss function

# Pretrained VGG16

In [11]:
base_model = tf.keras.applications.VGG19(
    weights='imagenet',  # Load weights pre-trained on ImageNet.
    input_shape=(256, 256, 3),
    include_top=False)  # Do not include the ImageNet classifier at the top.
base_model.trainable = False



In [12]:
inputImg = tf.keras.layers.Input((256,256,3) , name='image')
s = Lambda(lambda x: x / 255)(inputImg)   
x = base_model(inputImg,training=False)
#tf.keras.layers.Flatten()(model.layers[-1].output)
imageModelOutput = tf.keras.layers.GlobalAveragePooling2D()(x)
b1 = tf.keras.layers.BatchNormalization()(imageModelOutput)
d1 = tf.keras.layers.Dense(256, activation='relu' )(b1)
d1 = tf.keras.layers.Dense(512, activation='relu' )(d1)
d1 = tf.keras.layers.Dense(256, activation='relu' )(d1)
d1 = tf.keras.layers.Dense(512, activation='relu' )(d1)
d1 = tf.keras.layers.Dense(256, activation='relu' )(d1)
reshaped = tf.keras.layers.Reshape((16, 16,1), input_shape=(256,)) (d1)
c1Dec = tf.keras.layers.Conv2DTranspose(16, (3, 3),strides=(2, 2), activation='relu', padding="same") (reshaped)
c2Dec = tf.keras.layers.Conv2DTranspose(32, (3, 3),strides=(2, 2), activation='relu', padding="same") (c1Dec)
c3Dec = tf.keras.layers.Conv2DTranspose(64, (3, 3),strides=(2, 2), activation='relu', padding="same") (c2Dec)
c4Dec = tf.keras.layers.Conv2DTranspose(256, (3, 3),strides=(2, 2), activation='relu', padding="same") (c3Dec)
c4Dec = tf.keras.layers.Conv2DTranspose(512, (3, 3),strides=(2, 2), activation='relu', padding="same") (c3Dec)
c4Dec = tf.keras.layers.Conv2DTranspose(1024, (3, 3),strides=(2, 2), activation='relu', padding="same") (c3Dec)
c4Dec = tf.keras.layers.Conv2DTranspose(1024, (3, 3),strides=(2, 2), activation='relu', padding="same") (c3Dec)
finalOutput = tf.keras.layers.Conv2D(1, (1, 1),activation='sigmoid') (c4Dec)

model = tf.keras.Model(inputs=[inputImg], outputs=[finalOutput])
model.compile(optimizer=tf.keras.optimizers.Adam(0.001),
    loss=[jacard_coef_loss],
    metrics=[iou_coef])

model.summary()

# U-NET

In [13]:
import tensorflow as tf
from keras import backend as K
config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True
config.log_device_placement = True
session = tf.compat.v1.Session(config=config)
K.set_session(session)

In [14]:
'''
results = model.fit({'image': train_images, 'metadata' : train_metadata[["age_approximate", "sex"]][0:100].to_numpy(dtype=np.float)}, train_masks, batch_size=32, epochs=50, callbacks=callbacks,
                    validation_data=({ 'image' : val_images, 'metadata': val_metadata[["age_approximate", "sex"]][0:100].to_numpy(dtype=np.float)}, val_masks))
'''
prepared_images = preprocess_input(train_images)
#prepared_images = preprocess_input(train_masks)
prepared_images = preprocess_input(val_images)
#prepared_images = preprocess_input(val_masks)
results = model.fit({'image': train_images}, train_masks, batch_size=16, epochs=50, callbacks=callbacks,
                    validation_data=({ 'image' : val_images}, val_masks))

# Visualize results

In [15]:
loss = results.history['loss']
val_loss = results.history['val_loss']
epochs = range(len(loss))
plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()
plt.show()